In [1]:
!gdown https://zenodo.org/record/7265581/files/ThorstenVoice-Dataset_2022.10.zip
!unzip /content/ThorstenVoice-Dataset_2022.10.zip
!rm /content/ThorstenVoice-Dataset_2022.10.zip
!rm -r /content/__MACOSX

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: ThorstenVoice-Dataset_2022.10/wavs/ce13177811baf4b27208cae361feab14.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/be2e0d8ccbdeae7c2ea76252634b84e3.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/6210e90145768bb13542016f6e5e62c5.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/343e874b5020e0825ba56455fb1e0091.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/7633567b2215ad04864e5d375d2cc86b.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/b99c4995a756768aecabd500ecd618bf.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/39b786ffb69a365cd206316059a2f864.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/eea8ca176390ff23eb9aa57b0559cbe2.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/cef5100fba5424231ea983bf0a625a20.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/a5d197a2bcb029fc1bf318b1c26cf1a6.wav  
  inflating: ThorstenVoice-Dataset_2022.10/wavs/868209a0be7d7c4

In [2]:
!pip install TTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.7/736.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.1/581.1 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [1]:
import os
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
import torch.multiprocessing as mp
import torch

In [2]:
# Prüfen ob GPU verfügbar - Trainingsdauer für 50 Epochen ohne GPU > 10 Stunden
!nvidia-smi
print(torch.cuda.is_available())
print(torch.cuda.device_count())

Fri Jun  2 15:21:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# GPU Sichtbar machen, sofern GPU vorhanden
if torch.cuda.is_available():
    if torch.cuda.device_count() > 0: # Haupt GPU verwenden!
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
# Ausgabepfad für Traingslauf und Ergebnis
output_path = "/content/train"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [10]:
# Definieren eines Datasets. ljspeech ist ein definiertes Format für Datasets
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata_train.csv", path="/content/ThorstenVoice-Dataset_2022.10/"
)

In [11]:
# Konfiguration des TTS-Modells. Für Multithreading müssen die Anzahl Worker definiert werden, werden über CPU Cores ausgelesen
# Konfiguration weitesgehend aus dem "Tutorial for beginners" übernommen, "epochs" von 1000 auf 100 geändert.
cpu_cores = mp.cpu_count()
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=cpu_cores,
    num_eval_loader_workers=cpu_cores,
    precompute_num_workers=cpu_cores,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="de-de",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    test_sentences=[
        "Es hat mich viel Zeit gekostet eine Stimme zu entwickeln, jetzt wo ich sie habe werde ich nicht mehr schweigen.",
        "Sei eine Stimme, kein Echo.",
        "Es tut mir Leid David. Das kann ich leider nicht machen.",
        "Dieser Kuchen ist großartig. Er ist so lecker und feucht.",
        "Vor dem 22. November 1963.",
    ],
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

In [12]:
# Audio processor extrahiert features und verarbeitet Audio-Files
# SampleRate muss zu Trainingsdaten passen!
ap = AudioProcessor.init_from_config(config)
ap.sample_rate = 22050

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [13]:
# Tokenizer für Umwandlung von Text in token IDs.
tokenizer, config = TTSTokenizer.init_from_config(config)

In [14]:
# Testdaten laden, TTS bietet eine Funktion für die automatische Ermittlung von Verhältnis Training-/Testdaten
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 12283 files in /content/ThorstenVoice-Dataset_2022.10


In [15]:
# Modell intialisieren mit Config, AudioProcessor, Tokenizer un SpeakerManager
# SpeakerManager ermöglicht ein TTS-Modell mit mehreren Sprachen oder Sprechern
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [16]:
# Trainer wird ebenfalls geliefert, Parameter werden hier zusammengeführt:
# - Config
# - Ausgabeordner
# - Modell
# - Training und Testdatensätze

trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=/content/train/run-June-02-2023_03+24PM-0000000

 > Model has 28610257 parameters


In [ ]:
# Training starten
trainer.fit()


 > EPOCH: 0/100
 --> /content/train/run-June-02-2023_03+24PM-0000000


[*] Pre-computing phonemes...


  0%|          | 0/12161 [00:00<?, ?it/s]

daː mʊs aeːn vaeːnaxtsfɛst møːklɪç zaɪ̯n.
 [!] Character '̯' not found in the vocabulary. Discarding it.
das zɔl nuːn nɔɔʏ̯ə ʔaoːfɡaːbən bəkɔmən.
 [!] Character '̯' not found in the vocabulary. Discarding it.
zaɪ̯nə kaʁʁiːʁeː vaːɐ t͡suː ʔɛndeː, nɔx bəfoːɐ ziː bəɡɔnən hatə.
 [!] Character '͡' not found in the vocabulary. Discarding it.


  0%|          | 2/12161 [00:03<5:43:26,  1.69s/it]

...


   --> STEP: 14/381 -- GLOBAL_STEP: 2300
     | > loss: 1.76469  (1.75005)
     | > log_mle: 0.42173  (0.42592)
     | > loss_dur: 1.34296  (1.32413)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.94773  (3.87462)
     | > current_lr: 0.00000 
     | > step_time: 0.40950  (0.62810)
     | > loader_time: 0.00760  (0.00863)


   --> STEP: 39/381 -- GLOBAL_STEP: 2325
     | > loss: 1.68511  (1.71676)
     | > log_mle: 0.40076  (0.41611)
     | > loss_dur: 1.28435  (1.30066)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.84163  (3.85383)
     | > current_lr: 0.00000 
     | > step_time: 0.77850  (0.54804)
     | > loader_time: 0.01580  (0.00858)


   --> STEP: 64/381 -- GLOBAL_STEP: 2350
     | > loss: 1.64085  (1.68854)
     | > log_mle: 0.40619  (0.41240)
     | > loss_dur: 1.23466  (1.27614)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.75773  (3.81579)
     | > current_lr: 0.00000 
     | > step_time: 0.69410  (0.53



> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: de-de
		| > phoneme backend: gruut
	| > 2 not found characters:
	| > ̯
	| > ͡
| > Number of instances : 122
 | > Preprocessing samples
 | > Max text length: 98
 | > Min text length: 13
 | > Avg text length: 54.532786885245905
 | 
 | > Max audio length: 138506.0
 | > Min audio length: 24728.0
 | > Avg audio length: 71972.5655737705
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.02715 (+0.01988)
     | > avg_loss: 1.34975 (-0.27914)
     | > avg_log_mle: 0.33039 (-0.08006)
     | > avg_loss_dur: 1.01936 (-0.19908)

 > BEST MODEL : /content/train/run-June-02-2023_03+24PM-0000000/best_model_2667.pth

 > EPOCH: 7/100
 --> /content/train/run-June-02-2023_03+24PM-0000000

 > TRAINING (2023-06-02 16:22:21) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: de-de
		| > phoneme backend: gruut
	| > 2 not found characters:
	| > ̯
	| > ͡
| > Number of instances : 12161
 | > Preprocessing samples
 | > Max text length: 158
 | > Min text length: 6
 | > Avg text length: 54.56672970972782
 | 
 | > Max audio length: 190985.0
 | > Min audio length: 20318.0
 | > Avg audio length: 71807.7915467478
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 8/381 -- GLOBAL_STEP: 2675
     | > loss: 1.47876  (1.47551)
     | > log_mle: 0.35568  (0.34908)
     | > loss_dur: 1.12308  (1.12644)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.42425  (3.35908)
     | > current_lr: 0.00000 
     | > step_time: 0.38530  (0.39674)
     | > loader_time: 0.00340  (0.00556)


   --> STEP: 33/381 -- GLOBAL_STEP: 2700
     | > loss: 1.40122  (1.43560)
     | > log_mle: 0.33394  (0.33982)
     | > loss_dur: 1.06729  (1.09577)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.32942  (3.34171)
     | > current_lr: 0.00000 
     | > step_time: 0.49530  (0.53911)
     | > loader_time: 0.01100  (0.00751)


   --> STEP: 58/381 -- GLOBAL_STEP: 2725
     | > loss: 1.35280  (1.40631)
     | > log_mle: 0.33550  (0.33504)
     | > loss_dur: 1.01730  (1.07127)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.21911  (3.29778)
     | > current_lr: 0.00000 
     | > step_time: 1.18890  (0.567